## Data Preperation
After disccusion with Orit about the project and the time frame, we decied to-
1.leave just the Season '2019-2020'
2.count for each player how many teams and Leagues he played in this Season(collapse\group the data frame)

## Variables translation
GP	# Games Played
MIN	# Minutes Played
FGM	# Field Goals Made
FGA	# Field Goals Attempts
3PM	# Three Points Made
3PA	# Three Points Attempts
FTM	# Free Throws Made
FTA	# Free Throws Attempts
TOV	# Turnovers
PF	# Personal Fouls
ORB	# Offensive Rebounds
DRB	# Defensive Rebounds
REB	# Rebounds
AST	# Assists
STL	# Steals
BLK	# Blocks
PTS	# Points

In [131]:
import pandas as pd
import numpy as np
import uuid
import string
from datetime import datetime
import seaborn as sb

In [38]:
df = pd.read_csv('/Users/aslevi/Data Science-Bar Ilan /Basketball-ML Project/raw_players_stats_by_season_full_details.csv')

In [39]:

df.head()

,League,Season,Stage,Player,Team,GP,MIN,FGM,FGA,3PM,...,birth_date,height,height_cm,weight,weight_kg,nationality,high_school,draft_round,draft_pick,draft_team
0,NBA,1999 - 2000,Regular_Season,Shaquille O'Neal,LAL,79,3163.0,956,1665,0,...,"Mar 6, 1972",7-1,216.0,325.0,147.0,United States,Robert G. Cole High School,1.0,1.0,Orlando Magic
1,NBA,1999 - 2000,Regular_Season,Vince Carter,TOR,82,3126.0,788,1696,95,...,"Jan 26, 1977",6-6,198.0,220.0,100.0,United States,Mainland High School,1.0,5.0,Golden State Warriors
2,NBA,1999 - 2000,Regular_Season,Karl Malone,UTA,82,2947.0,752,1476,2,...,"Jul 24, 1963",6-9,206.0,265.0,120.0,United States,Summerfield High School,1.0,13.0,Utah Jazz
3,NBA,1999 - 2000,Regular_Season,Allen Iverson,PHI,70,2853.0,729,1733,89,...,"Jun 7, 1975",6-0,183.0,165.0,75.0,United States,Bethel High School,1.0,1.0,Philadelphia Sixers
4,NBA,1999 - 2000,Regular_Season,Gary Payton,SEA,82,3425.0,747,1666,177,...,"Jul 23, 1968",6-4,193.0,180.0,82.0,United States,Skyline High School,1.0,2.0,Seattle SuperSonics


In [40]:
# define the DF- Season '2019-2020'
df = df[df['Season']=='2019 - 2020']
df = df.drop('Season', axis=1)

In [41]:
df['birth_date']=df['birth_date'].replace(np.nan,'')
for ind in df.index:
        df['id'] = df['Player']+df['birth_date']

In [42]:
df['nba_played']= np.where(df['League']=='NBA',1,0)
df['playoff_played']= np.where((df['League']=='NBA') & (df['Stage']=='Playoffs'),1,0)

#df = df.drop(['League','Stage'], axis=1)

In [43]:
#Create unique ID for each player to make sure that we realized the specific player (in case we have players with the same name but different birth date)
df['unique_id'] = df.groupby(['id']).ngroup()
id_mapping = {group_id: uuid.uuid4().hex for group_id in df['unique_id'].unique()}
df['unique_id'] = df['unique_id'].map(id_mapping)
#df['unique_id'].value_counts()
#df[df['unique_id']==6277.0]

In [44]:
##How many players changed teams in the same Season(19-20)?- 67 Players
##How many players in this DF withot Team mention(N/A)?- 11 Players 
team_counts = df.groupby(['unique_id'])['Team'].nunique().reset_index()
#team_counts.info()
#len(team_counts[team_counts['Team']==0])
team_counts.sort_values(by='Team', ascending=False)
x = team_counts.groupby(['Team'])['unique_id'].nunique().reset_index()
x

,Team,unique_id
0,0,11
1,1,6246
2,2,67


In [ ]:
plt.figure(figsize=(10, 6))
x.plot(kind='bar', color='skyblue')

# Customize the plot
plt.title('Number of Players by Teams Played in a Season')
plt.xlabel('Number of Teams Played')
plt.ylabel('Number of Players')
plt.xticks(rotation=0)  # Keep x-axis labels horizontal
plt.grid(axis='y')  # Add grid lines on the y-axis

# Show the plot
plt.show()

In [ ]:
##How many players played in more than 1 League in this season?-729

league_counts = df.groupby('unique_id')['League'].nunique().reset_index()
len(league_counts[league_counts['League']>1])
league_counts[league_counts['League']>1]

y = league_counts.groupby('League')['unique_id'].nunique()
plt.figure(figsize=(10, 6))
y.plot(kind='bar', color='skyblue')

# Customize the plot
plt.title('Number of Players by Leagues Played in a Season')
plt.xlabel('Number of Teams Played')
plt.ylabel('Number of Leagues')
plt.xticks(rotation=0)  # Keep x-axis labels horizontal
plt.grid(axis='y')  # Add grid lines on the y-axis

# Show the plot
plt.show()
y

In [46]:
df['teams_each_player'] = df.groupby('unique_id')['Team'].transform('nunique')
df['leagues_each_player'] = df.groupby('unique_id')['League'].transform('nunique')
#df[df['Player']=='Melvin Johnson']

In [47]:
#df[df['leagues_each_player']>1]
collapse_columns = ['GP','MIN','FGM','FGA','3PM','3PA','FTM','FTA','TOV','PF','ORB','DRB','REB','AST','STL','BLK','PTS']
for col in collapse_columns:
    df[col] = df.groupby('unique_id')[col].transform('sum')

#df[df ['Player']=='Chris Johnson']

In [48]:
#convert players with this nationality strucure- 'first / second' to 'Two Nationalities'

for i in df.index:
    nationality = df.at[i, 'nationality']
    if isinstance(nationality, str) and '/' in nationality:
        df.at[i, 'nationality'] = 'Two Nationalities'
df['nationality'].value_counts()

nationality
United States             2030
Two Nationalities          537
Serbia                     289
China                      238
Germany                    180
                          ... 
South Sudan                  1
Zimbabwe                     1
Peru                         1
British Virgin Islands       1
Azerbaijan                   1
Name: count, Length: 88, dtype: int64

In [11]:
##need to verify with Orit what to do with this column
#df['high_school'].value_counts().to_csv('high.csv')

In [49]:
# Columns to convert
columns_to_convert = ['Player','birth_month','nationality','high_school','draft_team']

for col in columns_to_convert:
    df[col] = df[col].astype(str)
    df[col] = df[col].map(lambda x: x.translate(str.maketrans('', '', string.punctuation)))



In [50]:
#add age(today) column based on birth_date
#in addition, i decreased the age by 4 years-from 19-20 season  

# Step 1: Convert the birth_date column to datetime format
df['birth_date'] = pd.to_datetime(df['birth_date'], format='%b %d, %Y')

# Step 2: Calculate the age as a floating-point number
today = pd.to_datetime('today')

# Calculate the difference in years
df['age'] = today.year - df['birth_date'].dt.year

# Calculate the fraction of the year
df['age'] += ((today - df['birth_date']).dt.days % 365.25) / 365.25
df['age']-=(today.year -2020)


In [53]:
df = df.drop(columns=['Stage','League','Team','id','birth_date','height','high_school'],axis=1)
df = df.drop_duplicates()

In [68]:
df_position = pd.read_csv('/Users/aslevi/Data Science-Bar Ilan /Basketball-ML Project/players_stats_by_season_full_details-2019-2020.csv')

In [87]:
df_position = df_position[['Player','position']].drop_duplicates()
df = pd.merge(df,df_position,on='Player', how='left')
df

,Player,GP,MIN,FGM,FGA,3PM,3PA,FTM,FTA,TOV,...,draft_round,draft_pick,draft_team,nba_played,playoff_played,unique_id,teams_each_player,leagues_each_player,age,position
0,Kelsey Barlow,27,789.4,179,414,68,203,81,98,55,...,NaN,NaN,nan,0,0,fefc9789e91e4fc3b9b3e97506417ab2,1,1,29.514031,GF
1,Anthony Young,25,806.6,160,329,51,125,92,108,34,...,NaN,NaN,nan,0,0,6354c53dfc7f4835a8215d86cc95be6c,1,1,31.694045,C
2,Pedro Barral,25,759.8,153,331,33,93,102,131,78,...,NaN,NaN,nan,0,0,8ea081fb36ef4fd4a4698817ea5f66b4,1,1,26.834360,PG
3,Federico Mariani,24,856.3,120,277,44,112,135,151,55,...,NaN,NaN,nan,0,0,906da9fbdb0648d8affdb0c40005e87c,1,1,28.350445,F
4,Caio De Souza Pacheco,21,582.7,134,296,30,104,110,145,72,...,NaN,NaN,nan,0,0,cc33d7c5addd4e508aec7c7c4c3fac89,1,1,21.492129,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6473,Anuar Rahimshaikov,13,91.6,8,17,0,2,5,5,4,...,NaN,NaN,nan,0,0,46ae1416450d417f87b313b087864547,1,1,23.405886,SG
6474,Kyrylo Meshheryakov,15,127.0,7,28,2,13,3,4,5,...,NaN,NaN,nan,0,0,9495d3ff0f6649fb92fdc98863e7a6ad,1,1,25.010267,F
6475,Yaroslav Kadygrob,10,81.7,5,16,4,14,1,3,3,...,NaN,NaN,nan,0,0,21c3d427035f4c96b2eed4c4ddc8acd5,1,1,29.813142,G
6476,Ernesto Tkachuk,16,124.7,1,15,0,11,7,14,11,...,NaN,NaN,nan,0,0,c98a3ecd9078448a98a0f3097ef785c6,1,1,26.924709,PG


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6478 entries, 0 to 6477
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Player               6478 non-null   object 
 1   GP                   6478 non-null   int64  
 2   MIN                  6478 non-null   float64
 3   FGM                  6478 non-null   int64  
 4   FGA                  6478 non-null   int64  
 5   3PM                  6478 non-null   int64  
 6   3PA                  6478 non-null   int64  
 7   FTM                  6478 non-null   int64  
 8   FTA                  6478 non-null   int64  
 9   TOV                  6478 non-null   int64  
 10  PF                   6478 non-null   int64  
 11  ORB                  6478 non-null   int64  
 12  DRB                  6478 non-null   int64  
 13  REB                  6478 non-null   int64  
 14  AST                  6478 non-null   int64  
 15  STL                  6478 non-null   i

In [89]:
df.head(100)

,Player,GP,MIN,FGM,FGA,3PM,3PA,FTM,FTA,TOV,...,draft_round,draft_pick,draft_team,nba_played,playoff_played,unique_id,teams_each_player,leagues_each_player,age,position
0,Kelsey Barlow,27,789.4,179,414,68,203,81,98,55,...,NaN,NaN,nan,0,0,fefc9789e91e4fc3b9b3e97506417ab2,1,1,29.514031,GF
1,Anthony Young,25,806.6,160,329,51,125,92,108,34,...,NaN,NaN,nan,0,0,6354c53dfc7f4835a8215d86cc95be6c,1,1,31.694045,C
2,Pedro Barral,25,759.8,153,331,33,93,102,131,78,...,NaN,NaN,nan,0,0,8ea081fb36ef4fd4a4698817ea5f66b4,1,1,26.834360,PG
3,Federico Mariani,24,856.3,120,277,44,112,135,151,55,...,NaN,NaN,nan,0,0,906da9fbdb0648d8affdb0c40005e87c,1,1,28.350445,F
4,Caio De Souza Pacheco,21,582.7,134,296,30,104,110,145,72,...,NaN,NaN,nan,0,0,cc33d7c5addd4e508aec7c7c4c3fac89,1,1,21.492129,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Jose Defelippo,26,542.3,65,180,29,101,32,40,35,...,NaN,NaN,nan,0,0,2dedaa992dce461cb40fd32df1eb3e94,1,1,23.069131,SG
96,Jamaal Levy,24,603.6,73,130,16,49,29,52,30,...,NaN,NaN,nan,0,0,88dbf04f1b0149f28616831fe74ccd1b,1,1,37.615332,F
97,Martin Dario Leiva,26,540.0,78,142,0,4,32,88,48,...,NaN,NaN,nan,0,0,43aedd160f0c4abab8102278de70c2f9,1,1,40.325804,C
98,Lautaro Berra Fernandez,26,373.3,84,156,0,0,20,35,22,...,NaN,NaN,nan,0,0,66ef6ff26fde4b2db5c2ad9fd9729968,1,1,22.611910,C


In [119]:
##needs to clean all the incorrect values
##df[df['position']=='#15']
positions = ['G','F','PG','SG','SF','C','PF','GF','GF']
for i in df.index:
    position = df.at[i, 'position']
    
    # Check if position is a string before checking for '#'
    if isinstance(position, str) and '#' in position:
        df.at[i, 'position'] = None
    if isinstance(position, str) and '-' in position:
        df.at[i, 'position']=df.at[i, 'position'].replace('-','')

df['position'].value_counts()

position
G     1268
F     1120
PG     721
SG     660
SF     596
C      567
PF     560
GF     341
FC     234
Name: count, dtype: int64

In [129]:

## in cases we have no chance to add or query the position from other columns- we will create a new category(Other/NAN)

# Replace NaN values in the 'position' column with 'Other'
df['position'].fillna('Other', inplace=True)
df['position'].value_counts(dropna=False)

position
G        1268
F        1120
PG        721
SG        660
SF        596
C         567
PF        560
Other     411
GF        341
FC        234
Name: count, dtype: int64

#Pickle File

In [133]:
import pickle
#Write a pickle file
with open('Data_Preperation.pkl', 'wb') as f:
    pickle.dump(df, f)

print("income dataset saved as a pickle file.")

income dataset saved as a pickle file.


In [134]:
#Read a pickle file and load the data
#with open('Data_Preperation.pkl', 'rb') as file:
 #   loaded_data = pickle.load(file)